In [1]:
import numpy as np
import pandas as pd 
from elasticsearch import Elasticsearch
import sklearn.metrics as sl
import nltk
import spacy
from nltk.tokenize import word_tokenize

In [2]:
from gensim.parsing.preprocessing import remove_stopwords

In [8]:
from elasticsearch import helpers
client = Elasticsearch(
    "http://localhost:9200",
     basic_auth=("elastic", "elastic")

)
resp = client.info()

In [9]:
def precision(true:list, predect:list):
    f = 0
    for t in true :
        for p in predect:
            if p == t:
                f = f + 1
    return (f / len(predect))
def recall(true:list, predect:list):
    f = 0
    for t in true :
        for p in predect:
            if p == t:
                f = f + 1
    return f / len(true)

In [ ]:
def getDate():
    temp_id = 0
    temp_doc_id = []
    rel = {}
    with open('data\CISI.REL') as f:
        lines = ""
        for l in f.readlines():
            qid = l.strip().split()[0]
            if(qid == temp_id):
                temp_doc_id.append(l.strip().split()[1])
            else:
                rel[temp_id] = temp_doc_id
                temp_id = qid
                temp_doc_id = []
                temp_doc_id.append(l.strip().split()[1])


In [10]:
nlp = spacy.load("en_core_web_sm")
class QueryProcessor:

    def __init__(self, nlp, keep=None):
        self.nlp = nlp
        self.keep = keep or {'PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'}

    def generate_query(self, text):
        doc = self.nlp(text)
        query = ' '.join(token.text for token in doc if token.pos_ in self.keep)
        return query


In [22]:
class noprocess:
    def __init__(self, keep=None):
        self.truth =[]
        self.predect =[]
        self.rel = self.getRelData()
        self.qry = self.getQryData()
        self.keep = keep or {'PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'}


    def getRelData(self):
        temp_id = 0
        temp_doc_id = []
        rel = {}
        with open('data\CISI.REL') as f:
            lines = ""
            for l in f.readlines():
                qid = l.strip().split()[0]
                if(qid == temp_id):
                    temp_doc_id.append(l.strip().split()[1])
                else:
                    rel[temp_id] = temp_doc_id
                    temp_id = qid
                    temp_doc_id = []
                    temp_doc_id.append(l.strip().split()[1])
        return rel

    def getQryData(self):
        qry_list =[]
        qry_id = ""
        qry_text = ""
        with open('data\CISI.QRY') as f:
            qry = ""
            for l in f.readlines():
                qry += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
            qry = qry.lstrip("\n").split("\n")
        for l in qry:
            if l.startswith(".I"):
                qry_list.append({'id':qry_id, 'doc':qry_text.lstrip(" ")})
                qry_id = l.split(" ")[1].strip()
                qry_text = ""
            
            else:
                qry_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.
        qry_list.remove({'id':'' ,'doc':''})
        return qry_list


    def removeWh(self ,q):
        query = ''
        tokens =  word_tokenize(q)
        for word, tag in nltk.pos_tag(tokens):
            print(word,tag)
            if tag in  self.keep:
                query = ' '.join(word)
        print("qqq",query)
        return query





    def searchNoWhword(self,count, index):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        nlp = spacy.load("en_core_web_sm")
        for i in range(1,35):
            q = QueryProcessor(nlp).generate_query(self.qry[i]['doc'])
            query = {
                        "match": {
                            "doc": q
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index=index, query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            truth.append(self.rel[str(i+1)])
            pi = precision(self.rel[str(i+1)], ids)
            p = p + pi
            ri = recall(self.rel[str(i+1)], ids)
            r = r + ri
            f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return p / len(predect), r/len(predect), f1/len(predect)
    


    
    def searchNoSW(self,count,index):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        for i in range(1,35):
            q = remove_stopwords(self.qry[i]['doc'] )
            query = {
                        "match": {
                            "doc": q
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index=index, query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            truth.append(self.rel[str(i+1)])
            pi = precision(self.rel[str(i+1)], ids)
            p = p + pi
            ri = recall(self.rel[str(i+1)], ids)
            r = r + ri
            f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return p / len(predect), r/len(predect), f1/len(predect)
    

    def searchPR(self,count,index):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        for i in range(1,35):
            query = {
                        "match": {
                            "doc": self.qry[i]['doc'] 
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index=index, query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            truth.append(self.rel[str(i+1)])
            pi = precision(self.rel[str(i+1)], ids)
            p = p + pi
            ri = recall(self.rel[str(i+1)], ids)
            r = r + ri
            #print(pi,ri)
           # f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return p / len(predect), r/len(predect) , f1/len(predect)

nop = noprocess()

In [16]:
nop.searchPR(count=100, index= 'cisi')

(0.13647058823529412, 0.32433051372235, 0.0)

In [17]:
nop.searchNoSW(count=100, index= 'cisi')

(0.14029411764705885, 0.3392589128755711, 0.17114114617176152)

In [18]:
nop.searchNoWhword(count=100,  index= 'cisi')

(0.1423529411764706, 0.347762030237089, 0.17435062822288983)

In [23]:
nop.searchPR(count=100, index= 'anacisi')

(0.13647058823529412, 0.32433051372235, 0.0)

In [30]:
nop.searchNoSW(count=100, index= 'anacisi')

(0.14029411764705885, 0.3392589128755711, 0.17114114617176152)

In [31]:
nop.searchNoWhword(count=100,  index= 'anacisi')

(0.1423529411764706, 0.347762030237089, 0.17435062822288983)

In [13]:

nlp = spacy.load("en_core_web_sm")
class QueryProcessor:

    def __init__(self, nlp, keep=None):
        self.nlp = nlp
        self.keep = keep or {'PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'}

    def generate_query(self, text):
        doc = self.nlp(text)
        query = ' '.join(token.text for token in doc if token.pos_ in self.keep)
        return query


In [14]:
qp = QueryProcessor(nlp)
qp.generate_query('what is the song for caty perry')

'song caty perry'

In [19]:
doc_setting = {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_analyzer_keyword": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": [
                        "asciifolding",
                        "lowercase",
                        "synonym"
                    ]
                },
                "my_analyzer_shingle": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "asciifolding",
                        "lowercase",
                        "synonym"
                    ]
                }
            },
        }
    }, "mappings": {
   
            "properties": {
                'id':{
                    'type': "string"
                },
                "doc": {
                    "type": "text",
                    "index_analyzer": "my_analyzer_keyword",
                    "search_analyzer": "my_analyzer_shingle"
                }
            }
        
    }
}

In [20]:
client.index(index='anacisi', body=doc_setting)

<ipython-input-20-3d9b34b92592>:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.index(index='anacisi', body=doc_setting)


ObjectApiResponse({'_index': 'anacisi', '_id': 'qiLXWYEBraLnPera6o8W', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})